<a href="https://colab.research.google.com/github/450586509/practical-ml/blob/master/%E9%98%85%E8%AF%BB%E7%90%86%E8%A7%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install transformers

In [9]:
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import transformers
from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer, HfArgumentParser, SquadDataset
from transformers import SquadDataTrainingArguments as DataTrainingArguments
from transformers import Trainer, TrainingArguments

In [1]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
#!cp dev-v2.0.json train-v2.0.json


--2020-11-14 09:42:22--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       100%[===================>]   4.17M  --.-KB/s    in 0.1s    

2020-11-14 09:42:22 (43.3 MB/s) - ‘dev-v2.0.json’ saved [4370528/4370528]



In [10]:
logger = logging.getLogger(__name__)

In [11]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_fast: bool = field(default=False, metadata={"help": "Set this flag to use fast tokenization."})
    # If you want to tweak more attributes on your tokenizer, you should do it in a distinct script,
    # or just modify its tokenizer_config.json.
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )


In [12]:
sys.argv = ["run_squad.py", 
            "--model_type", "xlnet" , 
            "--model_name_or_path", "xlnet-large-cased",
            "--do_train", 
            "--do_eval",
            "--data_dir","./",
            "--version_2_with_negative",
            "--learning_rate", "3e-5",
            "--num_train_epochs","4",
            "--max_seq_length","384",
            "--doc_stride","128",
            "--output_dir","./wwm_cased_finetuned_squad/",
            "--per_gpu_eval_batch_size=2",
            "--per_gpu_train_batch_size=2",
            "--save_steps","5000"
            ]

In [13]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))

if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
    # If we pass only one argument to the script and it's the path to a json file,
    # let's parse it to get our arguments.
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
else:
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()

if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
):
    raise ValueError(
        f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )

In [14]:
print(data_args)

SquadDataTrainingArguments(model_type='xlnet', data_dir='./', max_seq_length=384, doc_stride=128, max_query_length=64, max_answer_length=30, overwrite_cache=False, version_2_with_negative=True, null_score_diff_threshold=0.0, n_best_size=20, lang_id=0, threads=1)


In [15]:
# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
)
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    training_args.local_rank,
    training_args.device,
    training_args.n_gpu,
    bool(training_args.local_rank != -1),
    training_args.fp16,
)
logger.info("Training/evaluation parameters %s", training_args)


11/14/2020 09:43:11 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
11/14/2020 09:43:11 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='./wwm_cased_finetuned_squad/', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=2, per_gpu_eval_batch_size=2, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=4.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov14_09-43-06_75954628c014', logging_first_step=False, logging_steps=500, save_steps=5000, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank

In [18]:
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
)
model = AutoModelForQuestionAnswering.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
)

Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForQuestionAnsweringSimple: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForQuestionAnsweringSimple from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForQuestionAnsweringSimple from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForQuestionAnsweringSimple were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
data_args

SquadDataTrainingArguments(model_type='xlnet', data_dir='./', max_seq_length=384, doc_stride=128, max_query_length=64, max_answer_length=30, overwrite_cache=False, version_2_with_negative=True, null_score_diff_threshold=0.0, n_best_size=20, lang_id=0, threads=1)

In [20]:
# Get datasets
is_language_sensitive = hasattr(model.config, "lang2id")
train_dataset = (
    SquadDataset(
        data_args, tokenizer=tokenizer, is_language_sensitive=is_language_sensitive, cache_dir=model_args.cache_dir
    )
    if training_args.do_train
    else None
)
eval_dataset = (
    SquadDataset(
        data_args,
        tokenizer=tokenizer,
        mode="dev",
        is_language_sensitive=is_language_sensitive,
        cache_dir=model_args.cache_dir,
    )
    if training_args.do_eval
    else None
)

11/14/2020 09:44:07 - INFO - filelock -   Lock 139742564156528 acquired on ./cached_train_XLNetTokenizer_384_v2.lock
11/14/2020 09:44:07 - INFO - filelock -   Lock 139742564156528 released on ./cached_train_XLNetTokenizer_384_v2.lock


EOFError: ignored

In [ ]:
# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Training
if training_args.do_train:
    trainer.train(
        model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
    )
    trainer.save_model()
    # For convenience, we also re-save the tokenizer to the same directory,
    # so that you can share your model easily on huggingface.co/models =)
    if trainer.is_world_master():
        tokenizer.save_pretrained(training_args.output_dir)